In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_splittest_split


ImportError: cannot import name 'train_test_splittest_split' from 'sklearn.model_selection' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\__init__.py)

SyntaxError: invalid syntax (<ipython-input-3-2ca62fba4cf5>, line 1)